
# 🍽️ Labmentix — Local Food Wastage Management System (All‑in‑One, SQLite + Streamlit)

This notebook does everything requested by Labmentix:

- Creates **SQLite** database & schema
- Loads the **4 CSVs** (providers, receivers, food_listings, claims)
- Demonstrates **CRUD** in SQL
- Runs **16 analysis queries** (15 required)
- **Exports a complete Streamlit web app (`app.py`)** so you can launch the site with `streamlit run app.py`

> Place this notebook and the four CSVs in the **same folder** and run the cells in order.


In [1]:

import os
import pandas as pd
from sqlalchemy import create_engine, text

DB_PATH = "food.db"
engine = create_engine(f"sqlite:///{DB_PATH}", echo=False, future=True)

print("Using SQLite DB at:", DB_PATH)


Using SQLite DB at: food.db


## 🧱 Create schema

In [2]:

schema_sql = """
DROP TABLE IF EXISTS claims;
DROP TABLE IF EXISTS food_listings;
DROP TABLE IF EXISTS receivers;
DROP TABLE IF EXISTS providers;

CREATE TABLE providers (
    Provider_ID   INTEGER PRIMARY KEY,
    Name          TEXT NOT NULL,
    Type          TEXT,
    Address       TEXT,
    City          TEXT,
    Contact       TEXT
);

CREATE TABLE receivers (
    Receiver_ID   INTEGER PRIMARY KEY,
    Name          TEXT NOT NULL,
    Type          TEXT,
    City          TEXT,
    Contact       TEXT
);

CREATE TABLE food_listings (
    Food_ID       INTEGER PRIMARY KEY,
    Food_Name     TEXT NOT NULL,
    Quantity      INTEGER NOT NULL,
    Expiry_Date   DATE NOT NULL,
    Provider_ID   INTEGER NOT NULL,
    Provider_Type TEXT,
    Location      TEXT,
    Food_Type     TEXT,
    Meal_Type     TEXT,
    FOREIGN KEY (Provider_ID) REFERENCES providers(Provider_ID) ON DELETE CASCADE
);

CREATE TABLE claims (
    Claim_ID     INTEGER PRIMARY KEY,
    Food_ID      INTEGER NOT NULL,
    Receiver_ID  INTEGER NOT NULL,
    Status       TEXT NOT NULL,
    Timestamp    DATETIME NOT NULL,
    FOREIGN KEY (Food_ID) REFERENCES food_listings(Food_ID) ON DELETE CASCADE,
    FOREIGN KEY (Receiver_ID) REFERENCES receivers(Receiver_ID) ON DELETE CASCADE
);

CREATE INDEX idx_providers_city ON providers(City);
CREATE INDEX idx_receivers_city ON receivers(City);
CREATE INDEX idx_food_location ON food_listings(Location);
CREATE INDEX idx_food_provider ON food_listings(Provider_ID);
CREATE INDEX idx_claims_food ON claims(Food_ID);
CREATE INDEX idx_claims_receiver ON claims(Receiver_ID);
CREATE INDEX idx_claims_status ON claims(Status);
"""

with engine.begin() as conn:
    for stmt in schema_sql.strip().split(";"):
        if stmt.strip():
            conn.exec_driver_sql(stmt + ";")
print("Schema created with indexes.")


Schema created with indexes.


## 📥 Load the four CSVs

In [3]:

providers_csv = "providers_data.csv"
receivers_csv = "receivers_data.csv"
food_csv      = "food_listings_data.csv"
claims_csv    = "claims_data.csv"

providers = pd.read_csv(providers_csv)
receivers = pd.read_csv(receivers_csv)
food_listings = pd.read_csv(food_csv)
claims = pd.read_csv(claims_csv)

# Clean types
if "Expiry_Date" in food_listings.columns:
    food_listings["Expiry_Date"] = pd.to_datetime(food_listings["Expiry_Date"]).dt.strftime("%Y-%m-%d")
if "Timestamp" in claims.columns:
    claims["Timestamp"] = pd.to_datetime(claims["Timestamp"]).dt.strftime("%Y-%m-%d %H:%M:%S")

display(providers.head(3))
display(receivers.head(3))
display(food_listings.head(3))
display(claims.head(3))

with engine.begin() as conn:
    providers.to_sql("providers", conn, if_exists="replace", index=False)
    receivers.to_sql("receivers", conn, if_exists="replace", index=False)
    food_listings.to_sql("food_listings", conn, if_exists="replace", index=False)
    claims.to_sql("claims", conn, if_exists="replace", index=False)

for t in ["providers","receivers","food_listings","claims"]:
    n = pd.read_sql(f"SELECT COUNT(*) AS n FROM {t}", engine).iloc[0,0]
    print(f"{t}: {n} rows")


,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297
2,3,Miller-Black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",Lake Jesusview,001-517-295-2206


,Receiver_ID,Name,Type,City,Contact
0,1,Donald Gomez,Shelter,Port Carlburgh,(955)922-5295
1,2,Laurie Ramos,Individual,Lewisburgh,761.042.1570
2,3,Ashley Mckee,NGO,South Randalltown,691-023-0094x856


,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,2025-03-17,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,2025-03-24,791,Grocery Store,West James,Non-Vegetarian,Dinner
2,3,Fruits,46,2025-03-28,478,Catering Service,Lake Regina,Vegan,Breakfast


,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,2025-03-05 05:26:00
1,2,353,391,Cancelled,2025-03-11 10:24:00
2,3,626,492,Completed,2025-03-21 00:59:00


providers: 1000 rows
receivers: 1000 rows
food_listings: 1000 rows
claims: 1000 rows


## ✍️ CRUD Demonstration (providers)

In [4]:

# CREATE
with engine.begin() as conn:
    conn.execute(text("""
        INSERT OR REPLACE INTO providers (Provider_ID, Name, Type, Address, City, Contact)
        VALUES (9999, 'Demo Provider', 'Restaurant', '123 Demo Street', 'Hyderabad', '999-999-9999')
    """))
pd.read_sql("SELECT * FROM providers WHERE Provider_ID=9999", engine)


,Provider_ID,Name,Type,Address,City,Contact
0,9999,Demo Provider,Restaurant,123 Demo Street,Hyderabad,999-999-9999


In [5]:

# UPDATE
with engine.begin() as conn:
    conn.execute(text("""
        UPDATE providers SET Contact='888-888-8888' WHERE Provider_ID=9999
    """))
pd.read_sql("SELECT * FROM providers WHERE Provider_ID=9999", engine)


,Provider_ID,Name,Type,Address,City,Contact
0,9999,Demo Provider,Restaurant,123 Demo Street,Hyderabad,888-888-8888


In [6]:

# DELETE
with engine.begin() as conn:
    conn.execute(text("""DELETE FROM providers WHERE Provider_ID=9999"""))
pd.read_sql("SELECT * FROM providers WHERE Provider_ID=9999", engine)


,Provider_ID,Name,Type,Address,City,Contact


## 📊 Analysis — 16 SQL Queries

In [7]:

from sqlalchemy import text
def run_query(sql, params=None):
    return pd.read_sql(text(sql), engine, params=params or {})


### 1. Providers per City

In [8]:
run_query('''
SELECT City, COUNT(*) AS total_providers
FROM providers
GROUP BY City
ORDER BY total_providers DESC''')

,City,total_providers
0,South Christopherborough,3
1,New Carol,3
2,Williamview,2
3,West Lauraborough,2
4,West Christopher,2
...,...,...
958,Alexanderchester,1
959,Aguirreville,1
960,Adamsville,1
961,Adamsview,1


### 2. Receivers per City

In [9]:
run_query('''
SELECT City, COUNT(*) AS total_receivers
FROM receivers
GROUP BY City
ORDER BY total_receivers DESC''')

,City,total_receivers
0,New Christopher,3
1,Williamsfort,2
2,West Richard,2
3,West James,2
4,Stevenmouth,2
...,...,...
961,Alexanderbury,1
962,Aguilarstad,1
963,Aguilarbury,1
964,Adamland,1


### 3. Provider type with most listings

In [10]:
run_query('''
SELECT COALESCE(p.Type, fl.Provider_Type) AS Provider_Type,
       COUNT(*) AS total_listings
FROM food_listings fl
LEFT JOIN providers p ON p.Provider_ID = fl.Provider_ID
GROUP BY COALESCE(p.Type, fl.Provider_Type)
ORDER BY total_listings DESC''')

,Provider_Type,total_listings
0,Supermarket,267
1,Restaurant,258
2,Grocery Store,243
3,Catering Service,232


### 4. Provider contacts by City (parameterized: Hyderabad)

In [11]:

df = run_query("""
SELECT Name, Contact, Type, Address
FROM providers
WHERE City = :city
ORDER BY Name
""", params={"city": "Hyderabad"})
df.head(20)


,Name,Contact,Type,Address


### 5. Top Receivers by total claims

In [12]:
run_query('''
SELECT r.Receiver_ID, r.Name, COUNT(*) AS total_claims
FROM receivers r
JOIN claims c ON r.Receiver_ID = c.Receiver_ID
GROUP BY r.Receiver_ID, r.Name
ORDER BY total_claims DESC''')

,Receiver_ID,Name,total_claims
0,276,Scott Hunter,5
1,371,William Frederick,5
2,742,Matthew Webb,5
3,800,Anthony Garcia,5
4,90,Alvin West,4
...,...,...,...
619,993,Alexis Hayes,1
620,994,Angelica Simmons,1
621,995,Brittany Taylor,1
622,999,Cheyenne Ramsey,1


### 6. Total non-expired quantity (as of today)

In [13]:
run_query('''
SELECT SUM(Quantity) AS total_available_quantity
FROM food_listings
WHERE DATE(Expiry_Date) >= DATE('now')''')

,total_available_quantity
0,None


### 7. Listings by City

In [14]:
run_query('''
SELECT Location AS City, COUNT(*) AS listings_count
FROM food_listings
GROUP BY Location
ORDER BY listings_count DESC''')

,City,listings_count
0,South Kathryn,6
1,New Carol,6
2,Perezport,5
3,Jimmyberg,5
4,East Angela,5
...,...,...
619,Andersonmouth,1
620,Amandashire,1
621,Allenborough,1
622,Alexanderchester,1


### 8. Most common Food Types

In [15]:
run_query('''
SELECT Food_Type, COUNT(*) AS occurrences
FROM food_listings
GROUP BY Food_Type
ORDER BY occurrences DESC''')

,Food_Type,occurrences
0,Vegetarian,336
1,Vegan,334
2,Non-Vegetarian,330


### 9. Claims per Food Item

In [16]:
run_query('''
SELECT fl.Food_ID, fl.Food_Name, COUNT(c.Claim_ID) AS total_claims
FROM food_listings fl
LEFT JOIN claims c ON c.Food_ID = fl.Food_ID
GROUP BY fl.Food_ID, fl.Food_Name
ORDER BY total_claims DESC, fl.Food_ID''')

,Food_ID,Food_Name,total_claims
0,463,Soup,5
1,486,Chicken,5
2,548,Fish,5
3,35,Rice,4
4,92,Chicken,4
...,...,...,...
995,987,Dairy,0
996,993,Soup,0
997,994,Salad,0
998,996,Fish,0


### 10. Top Providers by Completed Claims

In [17]:
run_query('''
SELECT p.Provider_ID, p.Name, COUNT(*) AS completed_claims
FROM providers p
JOIN food_listings fl ON fl.Provider_ID = p.Provider_ID
JOIN claims c ON c.Food_ID = fl.Food_ID
WHERE c.Status = 'Completed'
GROUP BY p.Provider_ID, p.Name
ORDER BY completed_claims DESC''')

,Provider_ID,Name,completed_claims
0,709,Barry Group,5
1,241,"Harper, Blake and Alexander",4
2,596,"Barnes, Castro and Curtis",4
3,752,Butler-Richardson,4
4,967,Miller Inc,4
...,...,...,...
246,983,"Munoz, Stone and Wallace",1
247,985,Frey PLC,1
248,995,Moore Group,1
249,998,Novak Group,1


### 11. % of Claims by Status

In [18]:
run_query('''
SELECT Status,
       COUNT(*) AS cnt,
       ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM claims), 2) AS pct
FROM claims
GROUP BY Status
ORDER BY cnt DESC''')

,Status,cnt,pct
0,Completed,339,33.9
1,Cancelled,336,33.6
2,Pending,325,32.5


### 12. Average quantity per Receiver (across their claims)

In [19]:
run_query('''
SELECT r.Receiver_ID, r.Name,
       ROUND(AVG(fl.Quantity), 2) AS avg_quantity_per_claim
FROM receivers r
JOIN claims c ON c.Receiver_ID = r.Receiver_ID
JOIN food_listings fl ON fl.Food_ID = c.Food_ID
GROUP BY r.Receiver_ID, r.Name
ORDER BY avg_quantity_per_claim DESC''')

,Receiver_ID,Name,avg_quantity_per_claim
0,99,Nancy Silva,50.0
1,282,Lisa Pitts,50.0
2,429,Daniel Williams,50.0
3,567,Peggy Knight,50.0
4,616,Christopher Wright,50.0
...,...,...,...
619,214,Corey Rodriguez,2.0
620,746,Kenneth Barnes,2.0
621,766,John Reynolds,2.0
622,765,Joshua Hooper,1.0


### 13. Most claimed Meal Type

In [20]:
run_query('''
SELECT fl.Meal_Type, COUNT(*) AS claims_count
FROM claims c
JOIN food_listings fl ON fl.Food_ID = c.Food_ID
GROUP BY fl.Meal_Type
ORDER BY claims_count DESC''')

,Meal_Type,claims_count
0,Breakfast,278
1,Lunch,250
2,Snacks,240
3,Dinner,232


### 14. Total quantity donated by Provider

In [21]:
run_query('''
SELECT p.Provider_ID, p.Name, SUM(fl.Quantity) AS total_quantity_donated
FROM providers p
JOIN food_listings fl ON fl.Provider_ID = p.Provider_ID
GROUP BY p.Provider_ID, p.Name
ORDER BY total_quantity_donated DESC''')

,Provider_ID,Name,total_quantity_donated
0,709,Barry Group,179
1,306,"Evans, Wright and Mitchell",158
2,655,Smith Group,150
3,315,Nelson LLC,142
4,678,Ruiz-Oneal,140
...,...,...,...
632,258,Gonzales-Moore,1
633,745,"Martinez, Armstrong and Carroll",1
634,849,Brown and Sons,1
635,870,Mcgee PLC,1


### 15. Near-expiry items within next N days (N=2)

In [22]:
run_query('''
SELECT Food_ID, Food_Name, Quantity, Expiry_Date, Location
FROM food_listings
WHERE DATE(Expiry_Date) BETWEEN DATE('now') AND DATE('now', '+2 day')
ORDER BY Expiry_Date ASC''')

,Food_ID,Food_Name,Quantity,Expiry_Date,Location


### 16. Unclaimed or not-yet-completed items

In [23]:
run_query('''
SELECT fl.Food_ID, fl.Food_Name, fl.Quantity, fl.Expiry_Date, fl.Location
FROM food_listings fl
LEFT JOIN claims c ON c.Food_ID = fl.Food_ID AND c.Status = 'Completed'
WHERE c.Claim_ID IS NULL
ORDER BY fl.Expiry_Date''')

,Food_ID,Food_Name,Quantity,Expiry_Date,Location
0,4,Fruits,15,2025-03-16,Kellytown
1,42,Rice,50,2025-03-16,North Michelle
2,48,Rice,30,2025-03-16,North Jamesberg
3,53,Fruits,13,2025-03-16,North Carolfurt
4,70,Rice,7,2025-03-16,Boylechester
...,...,...,...,...,...
708,936,Chicken,32,2025-03-30,Martinville
709,942,Rice,44,2025-03-30,New Leslieport
710,956,Chicken,39,2025-03-30,Heathermouth
711,996,Fish,15,2025-03-30,Phillipsfort


## 🌐 Export a full Streamlit web app (`app.py`)

In [24]:

app_code = r"""
import os
import datetime as dt
import pandas as pd
import streamlit as st
from sqlalchemy import create_engine, text

st.set_page_config(page_title="Local Food Wastage Management System", page_icon="🍽️", layout="wide")

DB_PATH = os.path.join(os.getcwd(), "food.db")
engine = create_engine(f"sqlite:///{DB_PATH}", future=True)

st.title("🍽️ Local Food Wastage Management System")
st.caption("Upload CSVs → Manage data (CRUD) → Analyze with 15+ SQL queries. (SQLite)")


# Ensure tables exist (idempotent)
with engine.begin() as conn:
    conn.exec_driver_sql(\"\"\"
    CREATE TABLE IF NOT EXISTS providers (
        Provider_ID   INTEGER PRIMARY KEY,
        Name          TEXT NOT NULL,
        Type          TEXT,
        Address       TEXT,
        City          TEXT,
        Contact       TEXT
    );
    \"\"\")
    conn.exec_driver_sql(\"\"\"
    CREATE TABLE IF NOT EXISTS receivers (
        Receiver_ID   INTEGER PRIMARY KEY,
        Name          TEXT NOT NULL,
        Type          TEXT,
        City          TEXT,
        Contact       TEXT
    );
    \"\"\")
    conn.exec_driver_sql(\"\"\"
    CREATE TABLE IF NOT EXISTS food_listings (
        Food_ID       INTEGER PRIMARY KEY,
        Food_Name     TEXT NOT NULL,
        Quantity      INTEGER NOT NULL,
        Expiry_Date   DATE NOT NULL,
        Provider_ID   INTEGER NOT NULL,
        Provider_Type TEXT,
        Location      TEXT,
        Food_Type     TEXT,
        Meal_Type     TEXT
    );
    \"\"\")
    conn.exec_driver_sql(\"\"\"
    CREATE TABLE IF NOT EXISTS claims (
        Claim_ID     INTEGER PRIMARY KEY,
        Food_ID      INTEGER NOT NULL,
        Receiver_ID  INTEGER NOT NULL,
        Status       TEXT NOT NULL,
        Timestamp    DATETIME NOT NULL
    );
    \"\"\")


tab1, tab2, tab3, tab4, tab5 = st.tabs([
    "📥 CSV Import", "📦 Food Listings (CRUD)", "👥 Providers & Receivers (CRUD)",
    "🧾 Claims (CRUD)", "📊 Analysis & Reports"
])

with tab1:
    st.subheader("Import CSVs")
    st.caption("Upload providers_data.csv, receivers_data.csv, food_listings_data.csv, claims_data.csv")

    files = st.file_uploader("Upload CSV(s)", type=["csv"], accept_multiple_files=True)
    if files:
        with engine.begin() as conn:
            for f in files:
                name = f.name.lower()
                df = pd.read_csv(f)
                if 'food' in name and 'list' in name:
                    if 'Expiry_Date' in df.columns:
                        df['Expiry_Date'] = pd.to_datetime(df['Expiry_Date']).dt.date
                    df.to_sql("food_listings", conn, if_exists="replace", index=False)
                    st.success(f"Loaded food_listings ({df.shape[0]} rows).")
                elif 'claim' in name:
                    if 'Timestamp' in df.columns:
                        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
                    df.to_sql("claims", conn, if_exists="replace", index=False)
                    st.success(f"Loaded claims ({df.shape[0]} rows).")
                elif 'provider' in name:
                    df.to_sql("providers", conn, if_exists="replace", index=False)
                    st.success(f"Loaded providers ({df.shape[0]} rows).")
                elif 'receiver' in name:
                    df.to_sql("receivers", conn, if_exists="replace", index=False)
                    st.success(f"Loaded receivers ({df.shape[0]} rows).")
                else:
                    st.warning(f"Unknown CSV: {f.name}")
        st.toast("CSV import completed.", icon="✅")

with tab2:
    st.subheader("Food Listings: View / Create / Update / Delete")

    with st.expander("View & Filter", expanded=True):
        city = st.text_input("Filter by City (Location)")
        meal = st.selectbox("Filter by Meal Type", ["", "Breakfast", "Lunch", "Dinner", "Snacks"])
        ftype = st.selectbox("Filter by Food Type", ["", "Vegetarian", "Non-Vegetarian", "Vegan"])
        base_q = "SELECT * FROM food_listings WHERE 1=1"
        params = {}
        if city:
            base_q += " AND Location = :city"
            params["city"] = city
        if meal:
            base_q += " AND Meal_Type = :meal"
            params["meal"] = meal
        if ftype:
            base_q += " AND Food_Type = :ftype"
            params["ftype"] = ftype
        df = pd.read_sql(text(base_q), engine, params=params)
        st.dataframe(df, use_container_width=True)

    st.markdown("---")
    st.markdown("**Create / Update / Delete**")
    colA, colB = st.columns(2)

    with colA:
        st.markdown("**Create New Listing**")
        with st.form("create_food"):
            Food_ID = st.number_input("Food_ID", min_value=1, step=1)
            Food_Name = st.text_input("Food_Name")
            Quantity = st.number_input("Quantity", min_value=1, step=1)
            Expiry_Date = st.date_input("Expiry_Date", value=dt.date.today())
            Provider_ID = st.number_input("Provider_ID", min_value=1, step=1)
            Provider_Type = st.text_input("Provider_Type")
            Location = st.text_input("Location")
            Food_Type = st.selectbox("Food_Type", ["Vegetarian", "Non-Vegetarian", "Vegan"])
            Meal_Type = st.selectbox("Meal_Type", ["Breakfast", "Lunch", "Dinner", "Snacks"])
            submitted = st.form_submit_button("Create")
        if submitted:
            with engine.begin() as conn:
                conn.execute(text(\"\"\"
                    INSERT INTO food_listings (Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location, Food_Type, Meal_Type)
                    VALUES (:Food_ID, :Food_Name, :Quantity, :Expiry_Date, :Provider_ID, :Provider_Type, :Location, :Food_Type, :Meal_Type)
                \"\"\"), dict(Food_ID=Food_ID, Food_Name=Food_Name, Quantity=Quantity, Expiry_Date=str(Expiry_Date),
                             Provider_ID=Provider_ID, Provider_Type=Provider_Type, Location=Location, Food_Type=Food_Type, Meal_Type=Meal_Type))
            st.success("Created.")

    with colB:
        st.markdown("**Update / Delete Listing**")
        edit_id = st.number_input("Target Food_ID", min_value=1, step=1, key="edit_food_id")
        with st.form("update_food"):
            new_qty = st.number_input("New Quantity", min_value=1, step=1, key="new_qty")
            new_exp = st.date_input("New Expiry_Date", value=dt.date.today(), key="new_exp")
            new_loc = st.text_input("New Location", key="new_loc")
            update_btn = st.form_submit_button("Update")
        if update_btn:
            with engine.begin() as conn:
                conn.execute(text(\"\"\"
                    UPDATE food_listings
                    SET Quantity=:q, Expiry_Date=:e, Location=:l
                    WHERE Food_ID=:id
                \"\"\"), dict(q=new_qty, e=str(new_exp), l=new_loc, id=edit_id))
            st.success("Updated.")

        if st.button("Delete Listing"):
            with engine.begin() as conn:
                conn.execute(text("DELETE FROM food_listings WHERE Food_ID=:id"), dict(id=edit_id))
            st.success("Deleted.")

with tab3:
    st.subheader("Providers & Receivers")
    col1, col2 = st.columns(2)

    with col1:
        st.markdown("**Providers**")
        pdf = pd.read_sql("SELECT * FROM providers", engine)
        st.dataframe(pdf, use_container_width=True)
        with st.form("prov_form"):
            Provider_ID = st.number_input("Provider_ID", min_value=1, step=1)
            Name = st.text_input("Name")
            Type = st.text_input("Type")
            Address = st.text_input("Address")
            City = st.text_input("City")
            Contact = st.text_input("Contact")
            action = st.selectbox("Action", ["Create", "Delete"])
            submit = st.form_submit_button("Submit")
        if submit:
            with engine.begin() as conn:
                if action == "Create":
                    df = pd.DataFrame([{
                        "Provider_ID": Provider_ID, "Name": Name, "Type": Type,
                        "Address": Address, "City": City, "Contact": Contact
                    }])
                    df.to_sql("providers", conn, if_exists="append", index=False)
                    st.success("Provider added.")
                else:
                    conn.execute(text("DELETE FROM providers WHERE Provider_ID=:id"), dict(id=Provider_ID))
                    st.success("Provider deleted.")

    with col2:
        st.markdown("**Receivers**")
        rdf = pd.read_sql("SELECT * FROM receivers", engine)
        st.dataframe(rdf, use_container_width=True)
        with st.form("recv_form"):
            Receiver_ID = st.number_input("Receiver_ID", min_value=1, step=1)
            NameR = st.text_input("Name")
            TypeR = st.text_input("Type")
            CityR = st.text_input("City")
            ContactR = st.text_input("Contact")
            actionR = st.selectbox("Action", ["Create", "Delete"])
            submitR = st.form_submit_button("Submit")
        if submitR:
            with engine.begin() as conn:
                if actionR == "Create":
                    df = pd.DataFrame([{
                        "Receiver_ID": Receiver_ID, "Name": NameR, "Type": TypeR,
                        "City": CityR, "Contact": ContactR
                    }])
                    df.to_sql("receivers", conn, if_exists="append", index=False)
                    st.success("Receiver added.")
                else:
                    conn.execute(text("DELETE FROM receivers WHERE Receiver_ID=:id"), dict(id=Receiver_ID))
                    st.success("Receiver deleted.")

with tab4:
    st.subheader("Claims")
    cdf = pd.read_sql("SELECT * FROM claims", engine)
    st.dataframe(cdf, use_container_width=True)

    with st.form("claim_form"):
        Claim_ID = st.number_input("Claim_ID", min_value=1, step=1)
        Food_ID = st.number_input("Food_ID", min_value=1, step=1)
        Receiver_ID = st.number_input("Receiver_ID", min_value=1, step=1)
        Status = st.selectbox("Status", ["Pending", "Completed", "Cancelled"])
        Timestamp = st.text_input("Timestamp (YYYY-MM-DD HH:MM:SS)")
        action = st.selectbox("Action", ["Create", "Update Status", "Delete"])
        submitC = st.form_submit_button("Submit")
    if submitC:
        with engine.begin() as conn:
            if action == "Create":
                conn.execute(text(\"\"\"
                    INSERT INTO claims (Claim_ID, Food_ID, Receiver_ID, Status, Timestamp)
                    VALUES (:Claim_ID, :Food_ID, :Receiver_ID, :Status, :Timestamp)
                \"\"\"), dict(Claim_ID=Claim_ID, Food_ID=Food_ID, Receiver_ID=Receiver_ID, Status=Status, Timestamp=Timestamp))
                st.success("Claim added.")
            elif action == "Update Status":
                conn.execute(text(\"\"\"
                    UPDATE claims SET Status=:Status, Timestamp=:Timestamp WHERE Claim_ID=:Claim_ID
                \"\"\"), dict(Status=Status, Timestamp=Timestamp, Claim_ID=Claim_ID))
                st.success("Claim updated.")
            else:
                conn.execute(text("DELETE FROM claims WHERE Claim_ID=:id"), dict(id=Claim_ID))
                st.success("Claim deleted.")

with tab5:
    st.subheader("Analysis & Reports (16 queries)")
    q_tabs = st.tabs([
        "1 Providers per City", "2 Receivers per City", "3 Provider Type Most Listings",
        "4 Provider Contacts by City", "5 Top Receivers by Claims",
        "6 Total Non-Expired Quantity", "7 Listings by City",
        "8 Common Food Types", "9 Claims per Food Item",
        "10 Top Providers by Completed Claims", "11 Claim Status %",
        "12 Avg Quantity per Receiver", "13 Most Claimed Meal Type",
        "14 Total Quantity by Provider", "15 Near-Expiry Items",
        "16 Unclaimed/Not Completed"
    ])

    with q_tabs[0]:
        df = pd.read_sql("SELECT City, COUNT(*) AS total_providers FROM providers GROUP BY City ORDER BY total_providers DESC", engine)
        st.dataframe(df, use_container_width=True)
        if not df.empty:
            st.bar_chart(df.set_index("City"))

    with q_tabs[1]:
        df = pd.read_sql("SELECT City, COUNT(*) AS total_receivers FROM receivers GROUP BY City ORDER BY total_receivers DESC", engine)
        st.dataframe(df, use_container_width=True)
        if not df.empty:
            st.bar_chart(df.set_index("City"))

    with q_tabs[2]:
        df = pd.read_sql(\"\"\"
            SELECT COALESCE(p.Type, fl.Provider_Type) AS Provider_Type,
                   COUNT(*) AS total_listings
            FROM food_listings fl
            LEFT JOIN providers p ON p.Provider_ID = fl.Provider_ID
            GROUP BY COALESCE(p.Type, fl.Provider_Type)
            ORDER BY total_listings DESC
        \"\"\", engine)
        st.dataframe(df, use_container_width=True)
        if not df.empty:
            st.bar_chart(df.set_index("Provider_Type"))

    with q_tabs[3]:
        city = st.text_input("City", key="contact_city")
        if city:
            df = pd.read_sql(text("SELECT Name, Contact, Type, Address FROM providers WHERE City=:city ORDER BY Name"), engine, params={"city": city})
            st.dataframe(df, use_container_width=True)

    with q_tabs[4]:
        df = pd.read_sql(\"\"\"
            SELECT r.Receiver_ID, r.Name, COUNT(*) AS total_claims
            FROM receivers r
            JOIN claims c ON r.Receiver_ID = c.Receiver_ID
            GROUP BY r.Receiver_ID, r.Name
            ORDER BY total_claims DESC
        \"\"\", engine)
        st.dataframe(df, use_container_width=True)

    with q_tabs[5]:
        df = pd.read_sql(\"\"\"
            SELECT SUM(Quantity) AS total_available_quantity
            FROM food_listings
            WHERE DATE(Expiry_Date) >= DATE('now')
        \"\"\", engine)
        st.metric("Total Non-Expired Quantity", int(df.iloc[0,0]) if not df.empty and pd.notna(df.iloc[0,0]) else 0)

    with q_tabs[6]:
        df = pd.read_sql(\"\"\"
            SELECT Location AS City, COUNT(*) AS listings_count
            FROM food_listings
            GROUP BY Location
            ORDER BY listings_count DESC
        \"\"\", engine)
        st.dataframe(df, use_container_width=True)
        if not df.empty:
            st.bar_chart(df.set_index("City"))

    with q_tabs[7]:
        df = pd.read_sql(\"\"\"
            SELECT Food_Type, COUNT(*) AS occurrences
            FROM food_listings
            GROUP BY Food_Type
            ORDER BY occurrences DESC
        \"\"\", engine)
        st.dataframe(df, use_container_width=True)
        if not df.empty:
            st.bar_chart(df.set_index("Food_Type"))

    with q_tabs[8]:
        df = pd.read_sql(\"\"\"
            SELECT fl.Food_ID, fl.Food_Name, COUNT(c.Claim_ID) AS total_claims
            FROM food_listings fl
            LEFT JOIN claims c ON c.Food_ID = fl.Food_ID
            GROUP BY fl.Food_ID, fl.Food_Name
            ORDER BY total_claims DESC, fl.Food_ID
        \"\"\", engine)
        st.dataframe(df, use_container_width=True)

    with q_tabs[9]:
        df = pd.read_sql(\"\"\"
            SELECT p.Provider_ID, p.Name, COUNT(*) AS completed_claims
            FROM providers p
            JOIN food_listings fl ON fl.Provider_ID = p.Provider_ID
            JOIN claims c ON c.Food_ID = fl.Food_ID
            WHERE c.Status = 'Completed'
            GROUP BY p.Provider_ID, p.Name
            ORDER BY completed_claims DESC
        \"\"\", engine)
        st.dataframe(df, use_container_width=True)

    with q_tabs[10]:
        df = pd.read_sql(\"\"\"
            SELECT Status,
                   COUNT(*) AS cnt,
                   ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM claims), 2) AS pct
            FROM claims
            GROUP BY Status
            ORDER BY cnt DESC
        \"\"\", engine)
        st.dataframe(df, use_container_width=True)

    with q_tabs[11]:
        df = pd.read_sql(\"\"\"
            SELECT r.Receiver_ID, r.Name,
                   ROUND(AVG(fl.Quantity), 2) AS avg_quantity_per_claim
            FROM receivers r
            JOIN claims c ON c.Receiver_ID = r.Receiver_ID
            JOIN food_listings fl ON fl.Food_ID = c.Food_ID
            GROUP BY r.Receiver_ID, r.Name
            ORDER BY avg_quantity_per_claim DESC
        \"\"\", engine)
        st.dataframe(df, use_container_width=True)

    with q_tabs[12]:
        df = pd.read_sql(\"\"\"
            SELECT fl.Meal_Type, COUNT(*) AS claims_count
            FROM claims c
            JOIN food_listings fl ON fl.Food_ID = c.Food_ID
            GROUP BY fl.Meal_Type
            ORDER BY claims_count DESC
        \"\"\", engine)
        st.dataframe(df, use_container_width=True)
        if not df.empty:
            st.bar_chart(df.set_index("Meal_Type"))

    with q_tabs[13]:
        df = pd.read_sql(\"\"\"
            SELECT p.Provider_ID, p.Name, SUM(fl.Quantity) AS total_quantity_donated
            FROM providers p
            JOIN food_listings fl ON fl.Provider_ID = p.Provider_ID
            GROUP BY p.Provider_ID, p.Name
            ORDER BY total_quantity_donated DESC
        \"\"\", engine)
        st.dataframe(df, use_container_width=True)

    with q_tabs[14]:
        days = st.number_input("Days Ahead", min_value=1, max_value=30, value=2, step=1)
        df = pd.read_sql(text(\"\"\"
            SELECT Food_ID, Food_Name, Quantity, Expiry_Date, Location
            FROM food_listings
            WHERE DATE(Expiry_Date) BETWEEN DATE('now') AND DATE('now', '+' || :days || ' day')
            ORDER BY Expiry_Date ASC
        \"\"\"), engine, params={"days": days})
        st.dataframe(df, use_container_width=True)

    with q_tabs[15]:
        df = pd.read_sql(\"\"\"
            SELECT fl.Food_ID, fl.Food_Name, fl.Quantity, fl.Expiry_Date, fl.Location
            FROM food_listings fl
            LEFT JOIN claims c ON c.Food_ID = fl.Food_ID AND c.Status = 'Completed'
            WHERE c.Claim_ID IS NULL
            ORDER BY fl.Expiry_Date
        \"\"\", engine)
        st.dataframe(df, use_container_width=True)
"""
with open("app.py","w",encoding="utf-8") as f:
    f.write(app_code)
print("Wrote app.py — run the web app with:  streamlit run app.py")


Wrote app.py — run the web app with:  streamlit run app.py



## ▶️ How to launch the Streamlit web app

1. Run all cells above (DB created + data loaded + `app.py` written).
2. In a terminal/shell from this notebook's folder:
   ```bash
   pip install streamlit pandas SQLAlchemy
   streamlit run app.py
   ```
3. In the app:
   - Go to **📥 CSV Import** to load/refresh data from your CSVs if needed.
   - Use **CRUD tabs** to manage data.
   - Explore **📊 Analysis & Reports** (16 built-in queries).
